In [4]:
import cv2
import numpy as np
# Function to count the fingers
def count_fingers(thresholded_image, hand_contour):
    # Find convex hull
    hull = cv2.convexHull(hand_contour, returnPoints=False)
    defects = cv2.convexityDefects(hand_contour, hull)

    if defects is None:
        return 0

    finger_count = 0
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        start = tuple(hand_contour[s][0])
        end = tuple(hand_contour[e][0])
        far = tuple(hand_contour[f][0])

        a = np.linalg.norm(np.array(end) - np.array(start))
        b = np.linalg.norm(np.array(far) - np.array(start))
        c = np.linalg.norm(np.array(end) - np.array(far))

        # Using cosine rule to find angle of the defect
        angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))

        # If angle is less than 90 degree, we count it as a finger
        if angle <= np.pi / 2:
            finger_count += 1

    return finger_count
# Initialize the video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip frame horizontally

   #Begin by creating a region of interest in a live video frame, where the hand is to be inserted for counting.
   # Define Region of Interest (ROI)
    roi = frame[100:400, 100:400]
    cv2.rectangle(frame, (100, 100), (400, 400), (0, 255, 0), 2)

    # Convert ROI to grayscale and apply Gaussian blur
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (35, 35), 0)

    # Once the hand is detected, it is isolated by applying thresholding techniques, Binary Thresholding in this case
    # using opencv.
    # Apply binary thresholding
    _, thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours and len(contours) > 0:
        hand_contour = max(contours, key=cv2.contourArea)

        # To the thresholded hand segment, a polygon is drawn around the hand to identify the extreme points, using
        # Convex Hull technique.
        # Find the convex hull and draw it
        hull = cv2.convexHull(hand_contour)
        cv2.drawContours(roi, [hull], -1, (0, 255, 0), 2)

        # Using the intersection of the extreme points in the polygon (Top, Bottom, Left, Right), the center of the hand is
        # calculated.
        # Find the center of the hand
        M = cv2.moments(hand_contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])

            # Draw the center of the hand
            cv2.circle(roi, (cX, cY), 7, (255, 255, 255), -1)

            # Calculate the maximum distance from the center to any point on the convex hull
            max_distance = max([np.linalg.norm(np.array([cX, cY]) - np.array(point[0])) for point in hull])

            # The point furthest away from the center of the hand is identified and a ratio of that distance say 80% is used as
            #the radius for a circle, which is drawn around the center of the hand (For visualization purposes, we may think of
            #this as the palm region).
            # Set the radius of the palm region as 80% of the maximum distance
            palm_radius = int(max_distance * 0.8)
            cv2.circle(roi, (cX, cY), palm_radius, (0, 0, 255), 2)

            ## Count the fingers
            finger_count = count_fingers(thresh, hand_contour)
            cv2.putText(frame, f"Fingers: {finger_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)
    cv2.imshow("Threshold", thresh)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
